# 🌍 LLM Translation Agent (Advanced)

This notebook demonstrates an **LLM-powered multilingual translation agent** with advanced features:  
- Multiple translation modes: literal, natural, creative  
- Tone control: neutral, formal, casual, professional, poetic  
- Translate **text, URLs, and PDFs**  
- Optional **explanation of translation choices**  
- Translate into **multiple languages at once**  
- Interactive interface with dropdowns and buttons  

This project showcases how LLMs can be used as **intelligent translation agents** rather than just direct machine translators.

## ⚙️ Setup

We will install the required Python libraries for translation, text extraction, and interactive widgets.

In [ ]:
!pip install openai requests PyPDF2 beautifulsoup4 ipywidgets

## 🧠 Translation Agent Class

The `LLMTranslationAgent` wraps the OpenAI LLM into a convenient interface.  

It provides methods for:  
- `translate()` → basic translation with tone & mode  
- `translate_url()` → fetch webpage text and translate  
- `translate_pdf()` → read a PDF and translate  
- `multi_translate()` → produce translations in multiple languages  

In [ ]:
import requests
from bs4 import BeautifulSoup
import PyPDF2
from openai import OpenAI

class LLMTranslationAgent:
    def __init__(self, model="gpt-4o-mini"):
        self.model = model
        self.client = OpenAI()

    def translate(self, text: str, target_lang: str = "French", mode: str = "natural", tone: str = "neutral", explain: bool = False) -> str:
        """
        Translate text using LLM with tone, style, and explanation options.

        :param text: Input text to translate.
        :param target_lang: Target language (e.g., "French", "Spanish").
        :param mode: "literal", "natural", or "creative".
        :param tone: "neutral", "formal", "casual", "professional", "poetic".
        :param explain: If True, provide translation notes.
        """
        instructions = {
            "literal": "Translate word-for-word as closely as possible.",
            "natural": "Translate naturally, as a fluent speaker would write.",
            "creative": "Translate with adaptation, localization, and stylistic creativity."
        }
        prompt = f"""
        Translate the following text into {target_lang}.
        Mode: {mode} — {instructions.get(mode, 'natural translation')}.
        Tone: {tone}.
        { 'Also explain translation choices, idioms, or cultural details.' if explain else '' }

        Text:
        {text}
        """

        resp = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5
        )
        return resp.choices[0].message.content.strip()

    def translate_url(self, url: str, **kwargs) -> str:
        """Fetch webpage text and translate."""
        html = requests.get(url).text
        soup = BeautifulSoup(html, "html.parser")
        text = " ".join(p.get_text() for p in soup.find_all("p"))
        return self.translate(text, **kwargs)

    def translate_pdf(self, file_path: str, **kwargs) -> str:
        """Extract text from PDF and translate."""
        with open(file_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            text = " ".join(page.extract_text() or "" for page in reader.pages)
        return self.translate(text, **kwargs)

    def multi_translate(self, text: str, target_langs: list, **kwargs) -> dict:
        """Translate into multiple languages at once."""
        results = {}
        for lang in target_langs:
            results[lang] = self.translate(text, target_lang=lang, **kwargs)
        return results

## 📝 Translation Prompt (used internally)

We guide the LLM with a clear instruction for consistent, context-aware translation:

```
You are an advanced multilingual translation assistant.  
Your goals:  
1. Translate text fluently and accurately into the requested target language.  
2. Respect the chosen mode:  
   - Literal → word-for-word accuracy.  
   - Natural → fluent, context-aware translation.  
   - Creative → localized, adapted, stylistically rich translation.  
3. Apply the requested tone (neutral, formal, casual, professional, poetic).  
4. If asked to explain, highlight tricky phrases, idioms, or cultural differences.  
5. Preserve key meaning while adapting to the requested style.  
```

## 🎛️ Interactive Translation Demo

Use the widgets below to test translations.  
- Enter some text  
- Select the target language  
- Choose a tone and mode  
- Optionally ask for explanations  
- Click **Translate**  

In [ ]:
import ipywidgets as widgets
from IPython.display import display

agent = LLMTranslationAgent()

text_input = widgets.Textarea(value="Hello, how are you?", placeholder="Enter text to translate...", description="Text:", layout=widgets.Layout(width="100%", height="100px"))
target_lang = widgets.Dropdown(options=["French", "Spanish", "German", "Chinese", "Arabic"], value="French", description="Target:")
tone = widgets.Dropdown(options=["neutral", "formal", "casual", "professional", "poetic"], value="neutral", description="Tone:")
mode = widgets.Dropdown(options=["literal", "natural", "creative"], value="natural", description="Mode:")
explain = widgets.Checkbox(value=False, description="Explain translation")

button = widgets.Button(description="Translate", button_style="success")
output = widgets.Output()

def on_translate_clicked(b):
    output.clear_output()
    with output:
        result = agent.translate(text_input.value, target_lang.value, mode.value, tone.value, explain.value)
        print("🔤 Translation:", result)

button.on_click(on_translate_clicked)
display(text_input, target_lang, tone, mode, explain, button, output)

## 🌐 Multi-Language Example

Here we translate the same phrase into **multiple languages at once**.  
This is useful when working in global teams or producing multilingual reports.

In [ ]:
agent = LLMTranslationAgent()
results = agent.multi_translate("The future of AI is promising.", ["French", "Spanish", "Chinese"], mode="natural")
for lang, trans in results.items():
    print(f"{lang}: {trans}")

## 🚀 Next Steps

You can extend this agent by:  
- Adding **speech-to-text** → translate spoken audio  
- Adding **text-to-speech** → listen to translations in native accents  
- Using **translation memory** → remember past translations for consistency  
- Deploying as a **web app** with Gradio/Streamlit  